In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
from src.visualize import *
from src.config import *
from src.util import *
pd.set_option('display.float_format', lambda x: '%.2f' % x)

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
texts_csv = '../input/feedback-prize-2021/texts.fea'
df_texts = pd.read_feather(texts_csv)
df_texts.head()

,index,id,text
0,0,4B22D00F9BD7,Students of the United States of America have ...
1,1,1F8DDCFCA766,"Have you ever awoken to the arrogant, angry sh..."
2,2,ED27B5020694,"The Battle for Control\n\nIn the near future, ..."
3,3,B1AF3AD74555,In the article the author talks about machines...
4,4,F0B1C389A815,Have you ever needed some dire advice for some...


In [3]:
df_texts['num_words'] = df_texts.text.apply(lambda x: len(x.split()))

In [4]:
df_texts.describe([.25,.5,.75,.9,.99,.999])

,index,num_words
count,15594.00,15594.00
mean,7796.50,421.26
std,4501.74,191.07
min,0.00,144.00
25%,3898.25,278.00
50%,7796.50,384.00
75%,11694.75,520.00
90%,14033.70,686.00
99%,15437.07,1016.00
99.9%,15577.41,1175.44


In [5]:
df_texts[df_texts['id']=='D753B3C6BDDA'].text.values[0].count('\n')

100

In [6]:
v = 44
# mn = 'mid.roberta-squad.sm=start'

In [7]:
gezi.init_flags()

In [8]:
# model_dir = f'../working/offline/{v}/0/{mn}'
model_dir = f'../working/offline/{v}'
info = gezi.load(f'{model_dir}/valid.pkl')
info.keys()

FileNotFoundError: [Errno 2] No such file or directory: '../working/offline/44/valid.pkl'

In [ ]:
# df_pred = get_preds(x, post_adjust=FLAGS.post_adjust)
# df_pred = link_evidence(df_pred)

In [ ]:
df_gt = pd.read_csv(f'{model_dir}/valid_gt.csv')
df_pred = pd.read_csv(f'{model_dir}/valid_pred.csv')

In [ ]:
def prepare(df, df_texts=None, fast=False):
  df['start'] = df.predictionstring.apply(lambda x: int(str(x).split()[0]))
  df.sort_values(['id', 'start'], inplace=True)
  df = df.reset_index(drop=True)
  if 'class' in df.columns:
    df.rename({'class': 'discourse_type'}, axis=1, inplace=True)
  if fast:
    return df
  if 'text' in df.columns:
    df.drop(['text'], axis=1, inplace=True)
  df = pd.merge(df, df_texts, how='left', on='id')
  df["essay_len"] = df["text"].apply(lambda x: len(x))
  df["essay_words"] = df["text"].apply(lambda x: len(x.split()))
  dis_starts, dis_ends = [], []
  text_list = []
  for row in tqdm(df.itertuples(), total=len(df), desc='vis_prepare1', leave=False):
    preds = row.predictionstring.split()
    s, e = int(preds[0]), int(preds[-1])
    texts = row.text.split()
    start = sum([len(x) + 1 for x in texts[:s]])
    end = sum([len(x) + 1 for x in texts[:e + 1]])
    dis_starts.append(start)
    dis_ends.append(end)
    text = ' '.join(texts)
    text_list.append(text)

  df['discourse_start'], df['discourse_end'] = dis_starts, dis_ends
  df['text'] = text_list

  df['gap_before'] = False
  df['gap_length'] = np.nan

  #set the first one
  df.loc[0, 'gap_before'] = True
  df.loc[0, 'gap_length'] = 0
  
  df['gap_end_length'] = df.essay_len - df.discourse_end

  #loop over rest
  for i in tqdm(range(1, len(df)), desc='vis_prepare2', leave=False):
    #gap if difference is not 1 within an essay
    if ((df.loc[i, "id"] == df.loc[i-1, "id"])\
        and (df.loc[i, "discourse_start"] - df.loc[i-1, "discourse_end"] > 0)):
      df.loc[i, 'gap_before'] = True
      df.loc[i,
             'gap_length'] = df.loc[i,
                                    "discourse_start"] - df.loc[i - 1,
                                                                "discourse_end"]
    #gap if the first discourse of an new essay does not start at 0
    elif ((df.loc[i, "id"] != df.loc[i-1, "id"])\
        and (df.loc[i, "discourse_start"] != 0)):
      df.loc[i, 'gap_before'] = True
      df.loc[i, 'gap_length'] = df.loc[i, "discourse_start"]

  return df

In [ ]:
df_gt = prepare(df_gt, df_texts)
df_pred = prepare(df_pred, df_texts)

vis_prepare1:   0%|          | 0/144293 [00:00<?, ?it/s]

vis_prepare2:   0%|          | 0/144292 [00:00<?, ?it/s]

vis_prepare1:   0%|          | 0/137368 [00:00<?, ?it/s]

vis_prepare2:   0%|          | 0/137367 [00:00<?, ?it/s]

In [ ]:
df_gt.head(2)

,level_0,index_x,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,...,text_,discourse_words,gap_before,gap_length,gap_end_length,discourse_type_id,kfold,num_words,index_y,text
0,59951,59951,0000D23A521A,1617734767734.00,0,170,"Some people belive that the so called ""face"" o...",Position,Position 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,...,"Some people belive that the so called ""face"" o...",34,True,0.00,1173,3,0,251,99,"Some people belive that the so called ""face"" o..."
1,59952,59952,0000D23A521A,1617734782429.00,170,357,"It was not created by aliens, and there is no ...",Evidence,Evidence 1,34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 4...,...,"Some people belive that the so called ""face"" o...",35,False,nan,986,2,0,251,99,"Some people belive that the so called ""face"" o..."


In [ ]:
l = []
ic(len(info['id']))
infos = {}
for i in tqdm(range(len(info['id']))):
  id = info['id'][i]
  m = {
    'id': info['id'][i],
    'start_probs': info['start_probs'][i][:,1],
    'probs': info['probs'][i],
    'preds': info['preds'][i]
  }
  infos[id] = m
  l.append(m) 
d = pd.DataFrame(l)
d

ic| <ipython-input-12-7b549c361283>:2 in <module>
    len(info['id']): 15594


  0%|          | 0/15594 [00:00<?, ?it/s]

,id,start_probs,probs,preds
0,0000D23A521A,"[0.28606481193087563, 0.00016431452167259562, ...","[[0.0031465774334546595, 0.0005460000944906803...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
1,00203C45FC55,"[0.1095542439099887, 0.0007105170399748789, 0....","[[0.00093990522070121, 0.0008884080641985265, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
2,0027FC00C35B,"[0.01012457054850018, 0.001643909220106461, 0....","[[0.9793871362500677, 0.002534964256767401, 0....","[0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
3,003CF65C2780,"[0.2983003112446299, 0.026230594955785693, 0.0...","[[0.030836693019114347, 0.0013687842098892673,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
4,004AC288D833,"[0.004916196756978211, 0.00012620873224900676,...","[[0.9967266029487862, 0.00031348658410261846, ...","[0, 0, 3, 3, 3, 3, 3, 3, 3, 1, 1, 0, 1, 1, 1, ..."
...,...,...,...,...
15589,FF7ADCC6146A,"[0.8113365192221603, 0.00047377420895714574, 0...","[[0.006370559157203906, 0.002540922558797743, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
15590,FF7E2F5C21CC,"[0.19175138065983924, 0.00150794988838485, 0.0...","[[0.009723777752182847, 0.0007278612741582868,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
15591,FF9DB9D43F72,"[0.20325322002703516, 0.0007575317558971834, 0...","[[0.005740019427871671, 0.0013193050396426013,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
15592,FFBA3014CDC7,"[0.11517521184020374, 0.0015696905791034764, 0...","[[0.005283370620790963, 0.000512157318425365, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."


In [ ]:
trans = pd.read_feather(f'../input/feedback-prize-2021/zh-cn.fea')

In [ ]:
trans.head()

,id,mid,para_type
0,0000D23A521A,[有些人认为火星上所谓的“脸”是由火星的生命创造的。不是这种情况。火星上的面孔是一种叫做梅萨...,"[3, 2, 2, 1, 6, 7, 2, 4]"
1,00066EA9880D,[无人驾驶汽车对他们所期望的东西更加激烈。在没有一个人的情况下驾驶的汽车实际上是在车轮上控制...,"[5, 3, 0, 1, 2, 1, 2, 0, 1, 2, 4]"
2,000E6DE9E817,"[[NONE], 我争论政策变革, [NONE], 尽管有一些孩子在那里真的需要帮助他们的学...","[0, 3, 0, 6, 7, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0, ..."
3,001552828BD0,[你能给你的车吗？不得不在公共汽车上骑陌生人来减少污染。驾驶汽车后，您能够支付罚款吗？[BR...,"[5, 2, 1, 1, 2, 1, 2, 1, 2, 3]"
4,0016926B079C,"[我认为学生将从家里学习，因为, 他们不会改变和, 早上起床淋浴，在那里做头发。, 只有课程...","[3, 1, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2]"


In [ ]:
df_gt[df_gt['id']=='D753B3C6BDDA'].text_.values[0].count('\n')

100

In [ ]:
trans[trans['id']=='D753B3C6BDDA'].mid.values[0]

array(['卢克的观点[BR] [BR]', '“来吧！请加入我！”我求。“这会很有趣！”[BR] [BR]',
       '“不，我不想。看起来很无聊。”萨姆说。[BR] [BR]“唐和我正在这样做。你想帮助有需要的人吗？”我说。[BR] [BR]“这件事怎么样，我会在我的父母那里谈论它，看看他们的想法，那么我会做出最终决定。好吗？”他解释道。[BR] [BR]“好的。明天在3岁的大树见到我。”我们双手走了。[BR] [BR]我真的希望Sam会同意。[BR] [BR]我们就像三个月斯科特！我想。[BR] [BR]唐，山姆，我是最好的朋友，因为我能记得。唐已经是我的邻居，所以当山姆在一对夫妇中搬到了众议院时，我们决定开始成为朋友的朋友。我很高兴我们做到了。[BR] [BR]唐，我现在一直待了一会儿，我们是一个刹车的家。我们唯一的使命虽然回家是试图让人们加入。我们已经有了一对夫妇，但我们需要更多。',
       '总有工作要在船上完成。[BR] [BR]',
       '“嗨妈妈。”我说走进厨房。 “晚餐闻起来很好。” [BR] [BR]“好。你今天过得怎么样？”她问。 “任何人都招募？” [BR] [BR]“仍然在山姆。他说他明天会告诉我他的最后答案。”我告诉她了。 [BR] [BR]我们吃了晚餐，然后清洗了菜肴。 [sep] [sep]“妈妈，如果我回到城里，它是好吗？”我问。 [BR] [BR]“当然。只是在黑暗之前回来。” [BR] [BR]我去了公共领域，因为这次在这时吃了很多人吃，和动物一起玩耍，或者小孩子们在跑去。我走进普通和肯定，人们在那里。 [BR] [BR]当我走路时，我觉得我的背部拍了一下。 [BR] [BR]“看起来我们有同样的想法，卢克！”这是唐。 [BR] [BR]“是的！很高兴你在这里。你有人得到了吗？”我问。 [BR] [BR]“是的。实际上。那些男人坐在那边说话。”他指着3个人在长凳上聊天。 [BR] [BR]“好工作！”我致意了他。 “山姆和我明天在大树见到他的最终决定。他说他今晚要和父母一起谈谈。” [sep] [sep]“哦，好。让我们去更多的人。”他回答。 [BR] [BR]“你好”。我对我这个年龄周围的一个男人抚摸着他的狗。 “我和未经撤退的小组，”海上牛仔“和我想知道你是否对此感兴趣。” [sep] [se

In [ ]:
def visulize(essay):
  y = df_gt[df_gt['id']==essay]
  y_ = df_pred[df_pred['id']==essay]
  res = {}
  f1s = []
  CLASSES = list(id2dis.values())[1:] 
  for c in CLASSES:
    try:
      ret = calc_f1(y[y['discourse_type'] == c],
                   y_[y_['discourse_type'] == c], return_dict=True)
      res[c] = ret['f1']
      # res[f'{c}/acc'] = ret['acc']
      # res[f'{c}/recall'] = ret['recall']
      f1s.append(ret['f1'])
    except Exception:
      pass
  res['Overall'] = np.mean(f1s)
  gezi.pprint_dict(res)
  visulize_df(df_gt, essay)
  if len(y_):
    visulize_df(df_pred, essay)
  text1 = y.text_.values[0]
  ENTER = '@@'
  text = text1.replace('\n', ENTER)
  l = []
  
  print(essay)
  text_cn_ = trans[trans['id']==essay]
  para_types = [id2dis[x] for x in text_cn_.para_type.values[0]]
  text_cn = list(zip(list(text_cn_.mid.values[0]), para_types))
  ic(text_cn)
  # print(text)
  
  words = text1.split()
  start_probs = infos[essay]['start_probs']
  start_probs = ['%.2f' % x for x in start_probs]
  token_probs = infos[essay]['probs']
  tokens = np.argmax(token_probs, -1)
  max_probs = token_probs.max(-1)
  max_probs = ['%.2f' % x for x in max_probs]
  # tokens = [id2dis[x] for x in tokens]
  print(list(zip(words, start_probs, tokens, max_probs)))
vis = visulize

In [ ]:
# B3DCC9D9797B 
# D753B3C6BDDA

In [ ]:
vis('0000D23A521A')


 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.4 |          0 |                      1 |      0 |              1 |          0 |    0.3429 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


0000D23A521A


ic| <ipython-input-17-15bf9090b021>:31 in visulize()
    text_cn: [('有些人认为火星上所谓的“脸”是由火星的生命创造的。不是这种情况。火星上的面孔是一种叫做梅萨的天然存在的地貌。', 'Position'),
              ('它不是由外星人创造的，并且没有阴谋隐藏在火星上隐藏外星生活。没有证据表明美国宇航局发现甚至认为这张脸是由外星人创造的。[BR] [BR]',
               'Evidence'),
              ('MESA是一个天然存在的岩层，在火星和地球上发现。', 'Evidence'),
              ('在火星上的这种“脸”看起来只看起来像一张脸，因为人类倾向于看到脸部，无论人类都在何处，人类都很非常非常社会，这就是为什么我们的大脑旨在识别面孔的原因。[BR] '
               '[BR]',
               'Claim'),
              ('许多阴谋理论家认为美国国家航空航天局在世界其他地区躲藏在火星上。', 'Counterclaim'),
              ('这些人会非常错误。如果美国国家航空航天局发现火星的生活，那么他们会得到数百万的人的关注。', 'Rebuttal'),
              ('美国宇航局的预算将增加剧烈，这意味着他们的工人会得到更多报酬。没有充分的理由，美国宇航局将在世界其他地区隐藏火星的生活。[BR] [BR]',
               'Evidence'),
              ('所以，美国国家航空航天局没有躲在美国火星上的生活，他们并不试图欺骗我们认为火星上的“脸”只是一个梅萨，因为它实际上是。美国宇航局隐藏生活将是不合逻辑的，因为如果他们在火星上找到了生活，他们会赚很多钱，我们都知道美国宇航局的人民不是不合逻辑的人。',
               'Concluding Statement')]


[('Some', '0.29', 5, '0.91'), ('people', '0.00', 5, '0.90'), ('belive', '0.00', 5, '0.91'), ('that', '0.00', 5, '0.90'), ('the', '0.00', 5, '0.90'), ('so', '0.00', 5, '0.90'), ('called', '0.00', 5, '0.90'), ('"face"', '0.00', 5, '0.90'), ('on', '0.00', 5, '0.90'), ('mars', '0.00', 5, '0.91'), ('was', '0.00', 5, '0.90'), ('created', '0.00', 5, '0.90'), ('by', '0.00', 5, '0.90'), ('life', '0.00', 5, '0.90'), ('on', '0.00', 5, '0.90'), ('mars.', '0.00', 5, '0.92'), ('This', '0.07', 5, '0.79'), ('is', '0.00', 5, '0.81'), ('not', '0.00', 5, '0.82'), ('the', '0.00', 5, '0.80'), ('case.', '0.00', 5, '0.80'), ('The', '0.82', 3, '0.90'), ('face', '0.00', 3, '0.91'), ('on', '0.00', 3, '0.91'), ('Mars', '0.00', 3, '0.91'), ('is', '0.00', 3, '0.91'), ('a', '0.00', 3, '0.91'), ('naturally', '0.00', 3, '0.90'), ('occuring', '0.00', 3, '0.90'), ('land', '0.00', 3, '0.92'), ('form', '0.00', 3, '0.92'), ('called', '0.00', 3, '0.91'), ('a', '0.00', 3, '0.92'), ('mesa.', '0.00', 3, '0.92'), ('It', '0.67'

In [ ]:
vis('D753B3C6BDDA')


 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|       0 |          0 |          0 |         0 |
+---------+------------+------------+-----------+


ic| <ipython-input-17-15bf9090b021>:31 in visulize()
    text_cn: [('卢克的观点[BR] [BR]', 'Nothing'),
              ('“来吧！请加入我！”我求。“这会很有趣！”[BR] [BR]', 'Position'),
              ('“不，我不想。看起来很无聊。”萨姆说。[BR] [BR]“唐和我正在这样做。你想帮助有需要的人吗？”我说。[BR] '
               '[BR]“这件事怎么样，我会在我的父母那里谈论它，看看他们的想法，那么我会做出最终决定。好吗？”他解释道。[BR] '
               '[BR]“好的。明天在3岁的大树见到我。”我们双手走了。[BR] [BR]我真的希望Sam会同意。[BR] '
               '[BR]我们就像三个月斯科特！我想。[BR] '
               '[BR]唐，山姆，我是最好的朋友，因为我能记得。唐已经是我的邻居，所以当山姆在一对夫妇中搬到了众议院时，我们决定开始成为朋友的朋友。我很高兴我们做到了。[BR] '
               '[BR]唐，我现在一直待了一会儿，我们是一个刹车的家。我们唯一的使命虽然回家是试图让人们加入。我们已经有了一对夫妇，但我们需要更多。',


D753B3C6BDDA


               'Nothing'),
              ('总有工作要在船上完成。[BR] [BR]', 'Claim'),
              ('“嗨妈妈。”我说走进厨房。 “晚餐闻起来很好。” [BR] [BR]“好。你今天过得怎么样？”她问。 “任何人都招募？” [BR] '
               '[BR]“仍然在山姆。他说他明天会告诉我他的最后答案。”我告诉她了。 [BR] [BR]我们吃了晚餐，然后清洗了菜肴。 [sep] '
               '[sep]“妈妈，如果我回到城里，它是好吗？”我问。 [BR] [BR]“当然。只是在黑暗之前回来。” [BR] '
               '[BR]我去了公共领域，因为这次在这时吃了很多人吃，和动物一起玩耍，或者小孩子们在跑去。我走进普通和肯定，人们在那里。 [BR] '
               '[BR]当我走路时，我觉得我的背部拍了一下。 [BR] [BR]“看起来我们有同样的想法，卢克！”这是唐。 [BR] '
               '[BR]“是的！很高兴你在这里。你有人得到了吗？”我问。 [BR] [BR]“是的。实际上。那些男人坐在那边说话。”他指着3个人在长凳上聊天。 '
               '[BR] [BR]“好工作！”我致意了他。 “山姆和我明天在大树见到他的最终决定。他说他今晚要和父母一起谈谈。” [sep] '
               '[sep]“哦，好。让我们去更多的人。”他回答。 [BR] [BR]“你好”。我对我这个年龄周围的一个男人抚摸着他的狗。 '
               '“我和未经撤退的小组，”海上牛仔“和我想知道你是否对此感兴趣。” [sep] [sep]“你好，并肯定。告诉我它。”他说，带着一种微笑。 [BR] '
               '[BR]',
               'Nothing'),
              ('“联合国救济和康复管理局以及我们所做的是什么是联合国救济和康复管理局，以船只对不同的地方。例如，我们去了希腊并卸下了。我们携带马，年轻奶牛和骡子。',
               'Evidence'),
         

[("Luke's", '0.04', 0, '0.95'), ('Point', '0.00', 0, '0.95'), ('Of', '0.00', 0, '0.95'), ('Veiw', '0.00', 0, '0.95'), ('"Come', '0.76', 0, '0.52'), ('on!', '0.02', 0, '0.29'), ('Please', '0.08', 3, '0.27'), ('join', '0.00', 3, '0.31'), ('with', '0.00', 3, '0.32'), ('me!"', '0.00', 3, '0.33'), ('I', '0.04', 5, '0.25'), ('begged.', '0.00', 0, '0.30'), ('"It', '0.10', 0, '0.32'), ('will', '0.02', 3, '0.32'), ('be', '0.00', 3, '0.33'), ('so', '0.00', 3, '0.32'), ('much', '0.00', 3, '0.31'), ('fun!"', '0.00', 3, '0.32'), ('"No.', '0.40', 0, '0.37'), ('I', '0.07', 6, '0.28'), ("don't", '0.00', 6, '0.29'), ('want', '0.00', 6, '0.29'), ('to.', '0.00', 6, '0.27'), ('It', '0.04', 6, '0.29'), ('seems', '0.00', 6, '0.31'), ('boring."', '0.00', 6, '0.31'), ('Sam', '0.04', 0, '0.31'), ('said.', '0.00', 0, '0.35'), ('"Don', '0.50', 0, '0.47'), ('and', '0.07', 7, '0.20'), ('I', '0.01', 7, '0.21'), ('are', '0.00', 7, '0.24'), ('doing', '0.00', 7, '0.22'), ('it.', '0.00', 7, '0.21'), ("Wouldn't", '0.20'

In [ ]:
ids = list(set(df_gt['id']))

In [ ]:
id = ids[random.randint(0, len(ids) - 1)]
ic(id)
vis(id)

ic| <ipython-input-34-dedd7e5b98d6>:2 in <module>- id: 'EC8FCCDB698A'



 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.3333 |        0.4 |          0 |    0.2444 |
+---------+------------+------------+-----------+


EC8FCCDB698A


ic| <ipython-input-17-15bf9090b021>:31 in visulize()
    text_cn: [('金星值得学习', 'Position'),
              ('[NONE]', 'Nothing'),
              ('当他们研究它时，他们称之为“傍晚的明星”，也是地球的“双胞胎”。', 'Claim'),
              ('这是地球上最近的行星，它也是最远的差距。维纳斯还拥有97％的二氧化碳毯，但表面上的800 [BR] [BR]华氏度。', 'Evidence'),
              ('自从', 'Nothing'),
              ('这个星球的条件就像那种对人类生活在那里的极端。', 'Claim'),
              ('维纳斯也具有我们太阳系中最热门的表面温度，即使你的水星最接近Sun Venus是较热的。它有腐蚀的火山，强大的地震和频率漏粉，在地球表面上发生。',
               'Evidence'),
              ('科学家认为，很久以前维纳斯就像地球，但即使现在它仍然拥有地球的一些功能。', 'Claim'),
              ('它有岩石沉积物，山谷，山脉，陨石坑。美国宇航局正在努力继续学习金星。他们已经完成了碳化硅测试，这些碳化硅测试是金星表面的混乱的模拟。',
               'Nothing'),
              ('其中一个是他们仍在努力满足金星所呈现的挑战。', 'Claim'),
              ('因此，如果有一种方法可以去那里并获得更多的样本，那么可能会告诉科学家可能在此之前有什么可能存在的东西，并在前面的线索中帮助他们出来的那样。他们也没有想到我们今天的计算机的快速。但他们还表示，只想想象暴露手机，平板电脑到酸，或者兼容金属锡，必须是金星上的热量。由于他们不能在金星上，只是轨道或安全地徘徊，他们无法获得他们想要的样本。',
               'Evidence')]


[('Venus', '0.73', 3, '0.94'), ('is', '0.00', 3, '0.92'), ('worth', '0.00', 3, '0.91'), ('studying', '0.00', 3, '0.93'), ('because', '0.71', 1, '0.37'), ('when', '0.24', 1, '0.54'), ('they', '0.00', 1, '0.57'), ('study', '0.00', 1, '0.57'), ('it', '0.00', 1, '0.56'), ('they', '0.00', 1, '0.59'), ('called', '0.00', 1, '0.57'), ('it', '0.00', 1, '0.57'), ('an', '0.00', 1, '0.57'), ('"Evening', '0.00', 1, '0.58'), ('Star"', '0.00', 1, '0.56'), ('also', '0.01', 1, '0.58'), ('a', '0.00', 1, '0.57'), ('"twin"', '0.00', 1, '0.56'), ('of', '0.00', 1, '0.57'), ('earth.', '0.00', 1, '0.55'), ("It's", '0.83', 2, '0.93'), ('the', '0.00', 2, '0.95'), ('closest', '0.00', 2, '0.95'), ('planet', '0.00', 2, '0.94'), ('to', '0.00', 2, '0.95'), ('earth,', '0.00', 2, '0.94'), ('It', '0.01', 2, '0.96'), ('is', '0.00', 2, '0.96'), ('the', '0.00', 2, '0.96'), ('closest', '0.00', 2, '0.96'), ('in', '0.00', 2, '0.96'), ('distence', '0.00', 2, '0.96'), ('as', '0.00', 2, '0.96'), ('well.', '0.00', 2, '0.96'), ('